Clean data scrapped from Octuparse

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
import re
import os
import itertools
from tqdm.auto import tqdm
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.io as pio
from typing import Any, Dict, List, Union

In [5]:
data_book_links = pd.read_excel("kindle/fixed_full_book_url_list v2-2 usable.xlsx")

In [21]:
data_book_links

,title,category_rank,format_kindle,book_info_link,located_category_page_link,located_category_page_title,Current_time,category,rank
0,"Da Vinci's Ghost: Genius, Obsession, and How L...",#1,Kindle Edition,https://www.amazon.com/Vincis-Ghost-Obsession-...,https://www.amazon.com/Best-Sellers-Kindle-Sto...,Amazon Best Sellers: Best Arts & Photography,2024-07-15 01:52:02.510,Arts & Photography,1
1,My Effin' Life,#2,Kindle Edition,https://www.amazon.com/My-Effin-Life-Geddy-Lee...,https://www.amazon.com/Best-Sellers-Kindle-Sto...,Amazon Best Sellers: Best Arts & Photography,2024-07-15 01:52:02.544,Arts & Photography,2
2,London Uncovered: Sixty Unusual Places to Explore,#3,Kindle Edition,https://www.amazon.com/London-Uncovered-Mark-D...,https://www.amazon.com/Best-Sellers-Kindle-Sto...,Amazon Best Sellers: Best Arts & Photography,2024-07-15 01:52:02.593,Arts & Photography,3
3,Brat: An '80s Story,#4,Kindle Edition,https://www.amazon.com/Brat-80s-Story-Andrew-M...,https://www.amazon.com/Best-Sellers-Kindle-Sto...,Amazon Best Sellers: Best Arts & Photography,2024-07-15 01:52:02.628,Arts & Photography,4
4,The Storyteller: Tales of Life and Music: A Me...,#5,Kindle Edition,https://www.amazon.com/Storyteller-Tales-Life-...,https://www.amazon.com/Best-Sellers-Kindle-Sto...,Amazon Best Sellers: Best Arts & Photography,2024-07-15 01:52:02.684,Arts & Photography,5
...,...,...,...,...,...,...,...,...,...
2880,Orca: Sailing Around the World (Five Oceans),#96,Kindle Edition,https://www.amazon.com/Orca-John-Pennington-eb...,https://www.amazon.com/Best-Sellers-Kindle-Sto...,Amazon Best Sellers: Best Travel,2024-07-15 02:10:32.665,Travel,96
2881,Neither here nor there: Travels in Europe,#97,Kindle Edition,https://www.amazon.com/Neither-here-nor-there-...,https://www.amazon.com/Best-Sellers-Kindle-Sto...,Amazon Best Sellers: Best Travel,2024-07-15 02:10:32.696,Travel,97
2882,Death in Yellowstone: Accidents and Foolhardin...,#98,Kindle Edition,https://www.amazon.com/Death-Yellowstone-Accid...,https://www.amazon.com/Best-Sellers-Kindle-Sto...,Amazon Best Sellers: Best Travel,2024-07-15 02:10:32.724,Travel,98
2883,FORTY NIGHTS IN ARUBA,#99,Kindle Edition,https://www.amazon.com/FORTY-NIGHTS-ARUBA-BEA-...,https://www.amazon.com/Best-Sellers-Kindle-Sto...,Amazon Best Sellers: Best Travel,2024-07-15 02:10:32.749,Travel,99


In [6]:
data_book_info = pd.read_excel("kindle/1dep_book_info_raw.xlsx")
data_book_info = data_book_info.drop(columns=["format"])
price_columns = ["kindle_selling_price", "kindle_original_price", "hardcover_list_price", "hardcover_selling_price", "paperback_selling_price", "paperback_original_price"]


def convert_price(price):
    try:
        return float(price.replace("US$", "").replace("$", ""))
    except ValueError:
        return None


for col in price_columns:
    data_book_info[col] = data_book_info[col].astype(str).apply(convert_price).astype("Float64")


def extract_kindle_rank(rank_str):
    match = re.search(r"暢銷商品排名:  #([\d,]+) 在 Kindle Store|Best Sellers Rank:  #([\d,]+) in Kindle Store", rank_str)
    if match:
        rank_num = match.group(1) or match.group(2)
        rank_num = rank_num.replace(",", "")
        return int(rank_num)
    return None


def extract_physical_rank(rank_str):
    match = re.search(r"暢銷商品排名:  #([\d,]+) 在 圖書", rank_str)
    if match:
        rank_num = match.group(1) or match.group(2)
        rank_num = rank_num.replace(",", "")
        return int(rank_num)
    return None


data_book_info["kindle-whole_rank"] = data_book_info["kindle_whole_rank"].astype(str).apply(extract_kindle_rank).astype("Float64")
data_book_info["paperback-whole_rank"] = data_book_info["paperback_info"].astype(str).apply(extract_physical_rank).astype("Float64")
data_book_info["hardcover-whole_rank"] = data_book_info["hardcover_info"].astype(str).apply(extract_physical_rank).astype("Float64")

data_book_info

,title,kindle_selling_price,kindle_original_price,kindle_whole_rank,kindle_num_page,kindle_publish_date,original_kindle_link,kindle_info,hardcover_list_price,hardcover_selling_price,hardcover_link,hardcover_info,paperback_selling_price,paperback_original_price,paperback_info,kindle-whole_rank,paperback-whole_rank,hardcover-whole_rank
0,"Da Vinci's Ghost: Genius, Obsession, and How L...",9.99,18.0,"暢銷商品排名: #60,324 在 Kindle Store (請參閱 Kindle St...",NaN,2012年 2月 7日,https://www.amazon.com/Vincis-Ghost-Obsession-...,產品詳細資訊\n \n ASIN\n ...,18.0,13.58,https://www.amazon.com/-/zh_TW/Toby-Lester/dp/...,產品詳細資訊\n \n ASIN\n ...,13.58,18.0,產品詳細資訊\n \n ASIN\n ...,60324.0,918176.0,918176.0
1,My Effin' Life,21.99,40.0,"暢銷商品排名: #12,518 在 Kindle Store (請參閱 Kindle St...",NaN,2023年 11月 14日,https://www.amazon.com/My-Effin-Life-Geddy-Lee...,產品詳細資訊\n \n ASIN\n ...,40.0,20.97,https://www.amazon.com/-/zh_TW/Geddy-Lee/dp/00...,產品詳細資訊\n \n 出版者\n ...,20.97,40.0,產品詳細資訊\n \n 出版者\n ...,12518.0,4007.0,4007.0
2,London Uncovered: Sixty Unusual Places to Explore,36.03,49.99,"Best Sellers Rank: #280,649 in Kindle Store (...",271 pages,"November 3, 2016",https://www.amazon.com/London-Uncovered-Mark-D...,Product details\n \n A...,50.0,37.93,https://www.amazon.com/London-Uncovered-New-Un...,產品詳細資訊\n \n 出版者\n ...,37.93,50.0,產品詳細資訊\n \n 出版者\n ...,280649.0,2084598.0,2084598.0
3,Brat: An '80s Story,11.99,18.99,"暢銷商品排名: #9,790 在 Kindle Store (請參閱 Kindle Sto...",NaN,2021年 5月 11日,https://www.amazon.com/Brat-80s-Story-Andrew-M...,產品詳細資訊\n \n ASIN\n ...,18.99,6.99,https://www.amazon.com/-/zh_TW/Andrew-McCarthy...,產品詳細資訊\n \n 出版者\n ...,6.99,18.99,產品詳細資訊\n \n 出版者\n ...,9790.0,12944.0,12944.0
4,The Storyteller: Tales of Life and Music: A Me...,2.99,21.99,"暢銷商品排名: #5,797 在 Kindle Store (請參閱 Kindle Sto...",NaN,2021年 10月 5日,https://www.amazon.com/Storyteller-Tales-Life-...,產品詳細資訊\n \n ASIN\n ...,21.99,14.47,https://www.amazon.com/-/zh_TW/Dave-Grohl/dp/0...,產品詳細資訊\n \n 出版者\n ...,14.47,21.99,產品詳細資訊\n \n 出版者\n ...,5797.0,8208.0,8208.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2868,Orca: Sailing Around the World (Five Oceans),3.99,<NA>,"暢銷商品排名: #72,952 在 Kindle Store (請參閱 Kindle St...",NaN,2014年 4月 29日,https://www.amazon.com/Orca-John-Pennington-eb...,產品詳細資訊\n \n ASIN\n ...,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,72952.0,<NA>,<NA>
2869,Neither here nor there: Travels in Europe,14.99,<NA>,"暢銷商品排名: #130,307 在 Kindle Store (請參閱 Kindle S...",NaN,2015年 6月 2日,https://www.amazon.com/Neither-here-nor-there-...,產品詳細資訊\n \n ASIN\n ...,18.99,6.99,https://www.amazon.com/-/zh_TW/Bill-Bryson/dp/...,NaN,6.99,18.99,NaN,130307.0,<NA>,<NA>
2870,Death in Yellowstone: Accidents and Foolhardin...,8.99,9.99,"暢銷商品排名: #70,687 在 Kindle Store (請參閱 Kindle St...",NaN,2014年 1月 7日,https://www.amazon.com/Death-Yellowstone-Accid...,產品詳細資訊\n \n ASIN\n ...,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,70687.0,<NA>,<NA>
2871,FORTY NIGHTS IN ARUBA,2.99,11.95,"暢銷商品排名: #75,871 在 Kindle Store (請參閱 Kindle St...",NaN,English,https://www.amazon.com/FORTY-NIGHTS-ARUBA-BEA-...,產品詳細資訊\n \n ASIN\n ...,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,75871.0,<NA>,<NA>


In [7]:
print(data_book_links.info())
print(data_book_info.info())
data_kindle_1dep = pd.merge(
    data_book_links[["title", "category", "book_info_link"]],
    data_book_info[["kindle_selling_price", "kindle_original_price", "kindle-whole_rank", "paperback_selling_price", "paperback_original_price", "paperback-whole_rank", "original_kindle_link"]],
    left_on="book_info_link",
    right_on="original_kindle_link",
    how="left",
)
print(data_kindle_1dep.info())
data_kindle_1dep

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2885 entries, 0 to 2884
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   title                        2885 non-null   object
 1   category_rank                2885 non-null   object
 2   format_kindle                2885 non-null   object
 3   book_info_link               2885 non-null   object
 4   located_category_page_link   2885 non-null   object
 5   located_category_page_title  2885 non-null   object
 6   Current_time                 2885 non-null   object
 7   category                     2885 non-null   object
 8   rank                         2885 non-null   int64 
dtypes: int64(1), object(8)
memory usage: 203.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2873 entries, 0 to 2872
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -

,title,category,book_info_link,kindle_selling_price,kindle_original_price,kindle-whole_rank,paperback_selling_price,paperback_original_price,paperback-whole_rank,original_kindle_link
0,"Da Vinci's Ghost: Genius, Obsession, and How L...",Arts & Photography,https://www.amazon.com/Vincis-Ghost-Obsession-...,9.99,18.0,60324.0,13.58,18.0,918176.0,https://www.amazon.com/Vincis-Ghost-Obsession-...
1,My Effin' Life,Arts & Photography,https://www.amazon.com/My-Effin-Life-Geddy-Lee...,21.99,40.0,12518.0,20.97,40.0,4007.0,https://www.amazon.com/My-Effin-Life-Geddy-Lee...
2,London Uncovered: Sixty Unusual Places to Explore,Arts & Photography,https://www.amazon.com/London-Uncovered-Mark-D...,36.03,49.99,280649.0,37.93,50.0,2084598.0,https://www.amazon.com/London-Uncovered-Mark-D...
3,Brat: An '80s Story,Arts & Photography,https://www.amazon.com/Brat-80s-Story-Andrew-M...,11.99,18.99,9790.0,6.99,18.99,12944.0,https://www.amazon.com/Brat-80s-Story-Andrew-M...
4,The Storyteller: Tales of Life and Music: A Me...,Arts & Photography,https://www.amazon.com/Storyteller-Tales-Life-...,2.99,21.99,5797.0,14.47,21.99,8208.0,https://www.amazon.com/Storyteller-Tales-Life-...
...,...,...,...,...,...,...,...,...,...,...
2880,Orca: Sailing Around the World (Five Oceans),Travel,https://www.amazon.com/Orca-John-Pennington-eb...,3.99,<NA>,72952.0,<NA>,<NA>,<NA>,https://www.amazon.com/Orca-John-Pennington-eb...
2881,Neither here nor there: Travels in Europe,Travel,https://www.amazon.com/Neither-here-nor-there-...,14.99,<NA>,130307.0,6.99,18.99,<NA>,https://www.amazon.com/Neither-here-nor-there-...
2882,Death in Yellowstone: Accidents and Foolhardin...,Travel,https://www.amazon.com/Death-Yellowstone-Accid...,8.99,9.99,70687.0,<NA>,<NA>,<NA>,https://www.amazon.com/Death-Yellowstone-Accid...
2883,FORTY NIGHTS IN ARUBA,Travel,https://www.amazon.com/FORTY-NIGHTS-ARUBA-BEA-...,2.99,11.95,75871.0,<NA>,<NA>,<NA>,https://www.amazon.com/FORTY-NIGHTS-ARUBA-BEA-...


In [8]:
data_kindle_1dep = data_kindle_1dep.rename(
    columns={
        "kindle_selling_price": "kindle-selling_price",
        "kindle_original_price": "kindle-original_price",
        "paperback_selling_price": "paperback-selling_price",
        "paperback_original_price": "paperback-original_price",
    }
)
data_kindle_1dep.head()

,title,category,book_info_link,kindle-selling_price,kindle-original_price,kindle-whole_rank,paperback-selling_price,paperback-original_price,paperback-whole_rank,original_kindle_link
0,"Da Vinci's Ghost: Genius, Obsession, and How L...",Arts & Photography,https://www.amazon.com/Vincis-Ghost-Obsession-...,9.99,18.0,60324.0,13.58,18.0,918176.0,https://www.amazon.com/Vincis-Ghost-Obsession-...
1,My Effin' Life,Arts & Photography,https://www.amazon.com/My-Effin-Life-Geddy-Lee...,21.99,40.0,12518.0,20.97,40.0,4007.0,https://www.amazon.com/My-Effin-Life-Geddy-Lee...
2,London Uncovered: Sixty Unusual Places to Explore,Arts & Photography,https://www.amazon.com/London-Uncovered-Mark-D...,36.03,49.99,280649.0,37.93,50.0,2084598.0,https://www.amazon.com/London-Uncovered-Mark-D...
3,Brat: An '80s Story,Arts & Photography,https://www.amazon.com/Brat-80s-Story-Andrew-M...,11.99,18.99,9790.0,6.99,18.99,12944.0,https://www.amazon.com/Brat-80s-Story-Andrew-M...
4,The Storyteller: Tales of Life and Music: A Me...,Arts & Photography,https://www.amazon.com/Storyteller-Tales-Life-...,2.99,21.99,5797.0,14.47,21.99,8208.0,https://www.amazon.com/Storyteller-Tales-Life-...


In [9]:
def fill_prices(row):
    for format in ["kindle", "paperback"]:
        selling_price_col = f"{format}-selling_price"
        original_price_col = f"{format}-original_price"

        if pd.isna(row[original_price_col]) and not pd.isna(row[selling_price_col]):
            row[original_price_col] = row[selling_price_col]
        elif not pd.isna(row[original_price_col]) and not pd.isna(row[selling_price_col]) and row[original_price_col] < row[selling_price_col]:
            row[original_price_col] = row[selling_price_col]

    return row


data_kindle_1dep = data_kindle_1dep.apply(fill_prices, axis=1)

In [26]:
data_kindle_1dep

,title,category,book_info_link,kindle-selling_price,kindle-original_price,kindle-whole_rank,paperback-selling_price,paperback-original_price,paperback-whole_rank,original_kindle_link
0,"Da Vinci's Ghost: Genius, Obsession, and How L...",Arts & Photography,https://www.amazon.com/Vincis-Ghost-Obsession-...,9.99,18.0,60324.0,13.58,18.0,918176.0,https://www.amazon.com/Vincis-Ghost-Obsession-...
1,My Effin' Life,Arts & Photography,https://www.amazon.com/My-Effin-Life-Geddy-Lee...,21.99,40.0,12518.0,20.97,40.0,4007.0,https://www.amazon.com/My-Effin-Life-Geddy-Lee...
2,London Uncovered: Sixty Unusual Places to Explore,Arts & Photography,https://www.amazon.com/London-Uncovered-Mark-D...,36.03,49.99,280649.0,37.93,50.0,2084598.0,https://www.amazon.com/London-Uncovered-Mark-D...
3,Brat: An '80s Story,Arts & Photography,https://www.amazon.com/Brat-80s-Story-Andrew-M...,11.99,18.99,9790.0,6.99,18.99,12944.0,https://www.amazon.com/Brat-80s-Story-Andrew-M...
4,The Storyteller: Tales of Life and Music: A Me...,Arts & Photography,https://www.amazon.com/Storyteller-Tales-Life-...,2.99,21.99,5797.0,14.47,21.99,8208.0,https://www.amazon.com/Storyteller-Tales-Life-...
...,...,...,...,...,...,...,...,...,...,...
2880,Orca: Sailing Around the World (Five Oceans),Travel,https://www.amazon.com/Orca-John-Pennington-eb...,3.99,3.99,72952.0,<NA>,<NA>,NaN,https://www.amazon.com/Orca-John-Pennington-eb...
2881,Neither here nor there: Travels in Europe,Travel,https://www.amazon.com/Neither-here-nor-there-...,14.99,14.99,130307.0,6.99,18.99,NaN,https://www.amazon.com/Neither-here-nor-there-...
2882,Death in Yellowstone: Accidents and Foolhardin...,Travel,https://www.amazon.com/Death-Yellowstone-Accid...,8.99,9.99,70687.0,<NA>,<NA>,NaN,https://www.amazon.com/Death-Yellowstone-Accid...
2883,FORTY NIGHTS IN ARUBA,Travel,https://www.amazon.com/FORTY-NIGHTS-ARUBA-BEA-...,2.99,11.95,75871.0,<NA>,<NA>,NaN,https://www.amazon.com/FORTY-NIGHTS-ARUBA-BEA-...


In [30]:
def plot_category_bar_plot(data: pd.DataFrame):
    category_counts = data["category"].value_counts().reset_index()
    category_counts.columns = ["category", "count"]

    # 繪製長條圖
    fig = px.bar(category_counts, x="category", y="count", labels={"category": "Category", "count": "Frequency"}, title="Distribution of Category 1dep in data_2dep_top100")

    # 顯示圖表
    fig.show()


plot_category_bar_plot(data_kindle_1dep)

In [35]:
def plot_knidle_paperback_price_scatter(data: pd.DataFrame):
    # 篩選掉缺少必要數據的行
    filtered_data = data.dropna(subset=["paperback-original_price", "kindle-original_price"])

    # 創建一個副本來調整價格
    adjusted_data = filtered_data.copy()
    # for fmt in ["paperback", "kindle"]:
    #     for pce in ["selling" , "original"]:
    adjusted_data[f"paperback-adjusted"] = adjusted_data["paperback-original_price"].apply(lambda x: min(x, 100))
    adjusted_data["kindle-adjusted"] = adjusted_data["kindle-original_price"].apply(lambda x: min(x, 100))

    # 設置軸的範圍和單位長
    min_val = 0
    max_val = 100
    buffer = 5  # 預留邊緣緩衝區
    min_val -= buffer
    max_val += buffer

    # 創建第一個子圖（未調整價格）
    fig1 = make_subplots(rows=1, cols=2, subplot_titles=("Original Price (Unadjusted)", "Selling Price (Unadjusted)"), x_title="Paperback Price", y_title="kindle Price")

    # 原價-未調整
    scatter1 = px.scatter(filtered_data, x="paperback-original_price", y="kindle-original_price")
    scatter1.update_traces(showlegend=False)
    fig1.add_traces(scatter1.data, rows=1, cols=1)

    # 售價-未調整
    scatter2 = px.scatter(filtered_data, x="paperback-selling_price", y="kindle-selling_price")
    scatter2.update_traces(showlegend=False)
    fig1.add_traces(scatter2.data, rows=1, cols=2)

    # 更新子圖佈局，設置軸範圍和單位長
    fig1.update_layout(
        height=600,
        width=1200,
        title_text="Comparison of Original and Selling Prices (Unadjusted)",
        xaxis1=dict(range=[min_val, max_val]),
        yaxis1=dict(range=[min_val, max_val]),
        xaxis2=dict(range=[min_val, max_val]),
        yaxis2=dict(range=[min_val, max_val]),
    )

    # 添加45度線
    for i in range(1, 3):
        fig1.add_trace(go.Scatter(x=[min_val, max_val], y=[min_val, max_val], mode="lines", line=dict(color="red", dash="dash"), showlegend=False), row=1, col=i)

    # 創建第二個子圖（調整價格）
    fig2 = make_subplots(rows=1, cols=2, subplot_titles=("Original Price", "Selling Price"), x_title="Paperback Price", y_title="kindle Price")

    # 原價-調整
    scatter3 = px.scatter(adjusted_data, x="paperback-adjusted", y="kindle-adjusted")
    scatter3.update_traces(showlegend=False)
    fig2.add_traces(scatter3.data, rows=1, cols=1)

    # 售價-調整
    scatter4 = px.scatter(adjusted_data, x="paperback-selling_price", y="kindle-selling_price")
    scatter4.update_traces(showlegend=False)
    fig2.add_traces(scatter4.data, rows=1, cols=2)

    # 更新子圖佈局，設置軸範圍和單位長
    fig2.update_layout(
        height=600,
        width=1200,
        title_text="Comparison of Original and Selling Prices",
        xaxis1=dict(range=[min_val, max_val]),
        yaxis1=dict(range=[min_val, max_val]),
        xaxis2=dict(range=[min_val, max_val]),
        yaxis2=dict(range=[min_val, max_val]),
    )

    # 添加45度線
    for i in range(1, 3):
        fig2.add_trace(go.Scatter(x=[min_val, max_val], y=[min_val, max_val], mode="lines", line=dict(color="red", dash="dash"), showlegend=False), row=1, col=i)

    fig1.show()
    fig2.write_image("0801 kindle-paperback/price scatter.svg")
    fig2.show()


plot_knidle_paperback_price_scatter(data_kindle_1dep)

In [10]:
data_kindle_1dep["kindle_over_paperback-selling_price"] = data_kindle_1dep["kindle-selling_price"] / data_kindle_1dep["paperback-selling_price"]
data_kindle_1dep["kindle_over_paperback-original_price"] = data_kindle_1dep["kindle-original_price"] / data_kindle_1dep["paperback-original_price"]

data_kindle_1dep["kindle-discount_rate"] = data_kindle_1dep["kindle-selling_price"] / data_kindle_1dep["kindle-original_price"]
data_kindle_1dep["paperback-discount_rate"] = data_kindle_1dep["paperback-selling_price"] / data_kindle_1dep["paperback-original_price"]

In [38]:
def plot_discount_rate_scatter(data: pd.DataFrame):
    filtered_data = data.dropna(subset=["paperback-discount_rate", "kindle-discount_rate"])

    # 設置軸的範圍和單位長
    min_val = 0
    max_val = max(filtered_data["paperback-discount_rate"].max(), filtered_data["kindle-discount_rate"].max())
    buffer = (max_val - min_val) * 0.05  # 預留邊緣緩衝區
    max_val += buffer

    # 創建散佈圖
    fig = px.scatter(
        filtered_data,
        x="paperback-discount_rate",
        y="kindle-discount_rate",
        labels={"paperback-discount_rate": "Paperback Discount Rate", "kindle-discount_rate": "Kindle Discount Rate"},
        title="Discount Rates for Paperback and kindle",
    )

    fig.update_layout(xaxis=dict(scaleanchor="y", scaleratio=1, range=[min_val, max_val]), yaxis=dict(range=[min_val, max_val]), width=600, height=600)  # 保證圖形是正方形

    # 添加45度線
    fig.add_trace(go.Scatter(x=[min_val, max_val], y=[min_val, max_val], mode="lines", line=dict(color="red", dash="dash"), showlegend=False))
    fig.write_image("0801 kindle-paperback/discount_rate_scatter.svg")
    fig.show()


plot_discount_rate_scatter(data_kindle_1dep)

In [52]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 篩選掉缺少必要數據的行
filtered_data = data_kindle_1dep.dropna(subset=["kindle_over_paperback-selling_price", "kindle_over_paperback-original_price"])
percentiles = [0.2, 0.4, 0.6, 0.8]
quantile_values = filtered_data["kindle_over_paperback-selling_price"].quantile(percentiles)
print(quantile_values)

# 設置軸的範圍和單位長
min_val = min(filtered_data["kindle_over_paperback-selling_price"].min(), filtered_data["kindle_over_paperback-original_price"].min())
max_val = max(filtered_data["kindle_over_paperback-selling_price"].max(), filtered_data["kindle_over_paperback-original_price"].max())
buffer = (max_val - min_val) * 0.10  # 預留邊緣緩衝區
min_val -= buffer
max_val += buffer

# 創建子圖
fig = make_subplots(rows=1, cols=2, subplot_titles=("Kindle Over Paperback Selling Price", "Kindle Over Paperback Original Price"), x_title="Ratio", y_title="Count")

# 售價直方圖
hist1 = px.histogram(filtered_data, x="kindle_over_paperback-selling_price", nbins=30, text_auto=True)
hist1.update_traces(showlegend=False)
fig.add_traces(hist1.data, rows=1, cols=1)

# 原價直方圖
hist2 = px.histogram(filtered_data, x="kindle_over_paperback-original_price", nbins=30, text_auto=True)
hist2.update_traces(showlegend=False)
fig.add_traces(hist2.data, rows=1, cols=2)

# 更新子圖佈局，設置軸範圍和單位長
fig.update_layout(height=600, width=1200, title_text="Histograms of Kindle Over Paperback Prices")

fig.update_xaxes(range=[min_val, max_val], row=1, col=1)
fig.update_xaxes(range=[min_val, max_val], row=1, col=2)
fig.update_yaxes(range=[0, 850], row=1, col=1)
fig.update_yaxes(range=[0, 850], row=1, col=2)
fig.write_image("0801 kindle-paperback/kindle_over_paperback_histogram_initial.svg")
fig.show()

0.2    0.321896
0.4    0.553138
0.6    0.769065
0.8    0.972184
Name: kindle_over_paperback-selling_price, dtype: object


In [16]:
filtered_data

,title,category,book_info_link,kindle-selling_price,kindle-original_price,kindle-whole_rank,paperback-selling_price,paperback-original_price,paperback-whole_rank,original_kindle_link,kindle_over_paperback-selling_price,kindle_over_paperback-original_price,kindle-discount_rate,paperback-discount_rate,selling_price_binned,original_price_binned
0,"Da Vinci's Ghost: Genius, Obsession, and How L...",Arts & Photography,https://www.amazon.com/Vincis-Ghost-Obsession-...,9.99,18.0,60324.0,13.58,18.0,918176.0,https://www.amazon.com/Vincis-Ghost-Obsession-...,0.735641,1.0,0.555,0.754444,"(0.5, 0.75]","(0.75, 1.0]"
1,My Effin' Life,Arts & Photography,https://www.amazon.com/My-Effin-Life-Geddy-Lee...,21.99,40.0,12518.0,20.97,40.0,4007.0,https://www.amazon.com/My-Effin-Life-Geddy-Lee...,1.048641,1.0,0.54975,0.52425,"(1.0, 1.25]","(0.75, 1.0]"
2,London Uncovered: Sixty Unusual Places to Explore,Arts & Photography,https://www.amazon.com/London-Uncovered-Mark-D...,36.03,49.99,280649.0,37.93,50.0,2084598.0,https://www.amazon.com/London-Uncovered-Mark-D...,0.949908,0.9998,0.720744,0.7586,"(0.75, 1.0]","(0.75, 1.0]"
3,Brat: An '80s Story,Arts & Photography,https://www.amazon.com/Brat-80s-Story-Andrew-M...,11.99,18.99,9790.0,6.99,18.99,12944.0,https://www.amazon.com/Brat-80s-Story-Andrew-M...,1.715308,1.0,0.631385,0.368088,"(1.5, 1.75]","(0.75, 1.0]"
4,The Storyteller: Tales of Life and Music: A Me...,Arts & Photography,https://www.amazon.com/Storyteller-Tales-Life-...,2.99,21.99,5797.0,14.47,21.99,8208.0,https://www.amazon.com/Storyteller-Tales-Life-...,0.206634,1.0,0.135971,0.658026,"(0.0, 0.25]","(0.75, 1.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2852,Born to Run,Travel,https://www.amazon.com/Born-Run-Christopher-Mc...,12.99,19.0,50712.0,11.44,19.0,<NA>,https://www.amazon.com/Born-Run-Christopher-Mc...,1.13549,1.0,0.683684,0.602105,"(1.0, 1.25]","(0.75, 1.0]"
2874,The Best Strangers in the World: Stories from ...,Travel,https://www.amazon.com/Best-Strangers-World-St...,12.99,18.99,84436.0,17.32,18.99,<NA>,https://www.amazon.com/Best-Strangers-World-St...,0.75,1.0,0.684044,0.912059,"(0.5, 0.75]","(0.75, 1.0]"
2876,Britain's Last Frontier: A Journey Along the H...,Travel,https://www.amazon.com/Britains-Last-Frontier-...,2.99,17.99,112923.0,5.0,5.0,<NA>,https://www.amazon.com/Britains-Last-Frontier-...,0.598,3.598,0.166203,1.0,"(0.5, 0.75]","(3.5, 4.0]"
2878,The Philosopher and the Druids: A Journey Amon...,Travel,https://www.amazon.com/Philosopher-Druids-Jour...,12.99,19.99,245907.0,14.43,14.43,<NA>,https://www.amazon.com/Philosopher-Druids-Jour...,0.900208,1.385308,0.649825,1.0,"(0.75, 1.0]","(1.25, 1.5]"


In [11]:
# 篩選掉缺少必要數據的行
filtered_data = data_kindle_1dep.dropna(subset=["kindle_over_paperback-selling_price", "kindle_over_paperback-original_price"])

# 定義區間範圍
bins = [0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2, 2.5, 3, 3.5, 4, float("inf")]

# 根據區間範圍計算頻率
filtered_data["selling_price_binned"] = pd.cut(filtered_data["kindle_over_paperback-selling_price"], bins=bins)
filtered_data["original_price_binned"] = pd.cut(filtered_data["kindle_over_paperback-original_price"], bins=bins)

selling_price_counts = filtered_data["selling_price_binned"].value_counts().sort_index()
original_price_counts = filtered_data["original_price_binned"].value_counts().sort_index()

# 設置區間標籤
labels = ["0-0.25", "0.25-0.5", "0.5-0.75", "0.75-1.0", "1.0-1.25", "1.25-1.5", "1.5-1.75", "1.75-2.0", "2.0-2.5", "2.5-3.0", "3.0-3.5", "3.5-4.0", ">4"]

# 創建子圖
fig = make_subplots(rows=1, cols=2, subplot_titles=("電子售價 / 平裝售價", "電子定價 / 平裝定價"), x_title="", y_title="", column_widths=[0.5, 0.5])

# 售價長條圖
fig.add_trace(go.Bar(x=labels, y=selling_price_counts.values, name="Selling Price", marker_color="#1f77b4"), row=1, col=1)

# 原價長條圖
fig.add_trace(go.Bar(x=labels, y=original_price_counts.values, name="Original Price", marker_color="#1f77b4"), row=1, col=2)  # 設置原價長條圖的顏色為藍色

# 設置相同的刻度範圍
max_y = max(selling_price_counts.max(), original_price_counts.max())
buffer = max_y * 0.1  # 預留邊緣緩衝區

fig.update_yaxes(title_font=dict(size=20), tickfont=dict(size=20), range=[0, max_y + buffer], row=1, col=1)
fig.update_yaxes(title_font=dict(size=20), tickfont=dict(size=20), range=[0, max_y + buffer], row=1, col=2)
fig.update_xaxes(tickfont=dict(size=16), row=1, col=1, title_standoff=100)  # 左子圖
fig.update_xaxes(tickfont=dict(size=16), row=1, col=2, title_standoff=100)  # 右子圖
fig.update_annotations(font=dict(size=20))

# 更新子圖佈局
fig.update_layout(height=700, width=1400, title_text="Bar Charts of Kindle Over Paperback Prices", showlegend=False)
fig.update_layout(font=dict(family=r"C:\Users\USER\OneDrive\桌面\Fonts\TaipeiSansTCBeta-Regular.ttf", color="Black"))
fig.update_layout(margin=dict(l=150, r=150, t=100, b=150))

fig.write_image("0801 kindle-paperback/kindle_over_paperback_histogram.svg")
fig.show()

C:\Users\USER\AppData\Local\Temp\ipykernel_26780\1122067092.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["selling_price_binned"] = pd.cut(filtered_data["kindle_over_paperback-selling_price"], bins=bins)
C:\Users\USER\AppData\Local\Temp\ipykernel_26780\1122067092.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["original_price_binned"] = pd.cut(filtered_data["kindle_over_paperback-original_price"], bins=bins)


In [12]:
# 創建 DataFrame 儲存售價區間頻率
selling_price_data = pd.DataFrame({"Selling Price Binned": selling_price_counts.index, "Selling Price Count": selling_price_counts.values})

# 創建 DataFrame 儲存原價區間頻率
original_price_data = pd.DataFrame({"Original Price Binned": original_price_counts.index, "Original Price Count": original_price_counts.values})

# 匯出為 Excel，分別放在不同的工作表中
with pd.ExcelWriter("1125/kindle_over_paperback_price_counts.xlsx") as writer:
    selling_price_data.to_excel(writer, sheet_name="Selling Price Data", index=False)
    original_price_data.to_excel(writer, sheet_name="Original Price Data", index=False)

In [17]:
data_books_com = pd.read_csv(r"C:\Users\USER\OneDrive\桌面\C2L2\data_bookscom\售價.csv")
filtered_data = data_books_com.dropna(subset=["kindle_over_paperback-selling_price", "kindle_over_paperback-original_price"])
filtered_data.head()

,kindle_over_paperback-original_price,kindle_over_paperback-selling_price
0,1.585236,0.798507
1,1.586001,0.798122
2,1.434783,0.776371
3,1.429968,0.778986
4,1.429536,0.779221


In [13]:
# 篩選掉缺少必要數據的行
data_books_com = pd.read_csv(r"C:\Users\USER\OneDrive\桌面\C2L2\data_bookscom\bookscom.csv")
filtered_data = data_books_com.dropna(subset=["kindle_over_paperback-selling_price", "kindle_over_paperback-original_price"])

# 定義區間範圍
bins = [0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2, 2.5, 3, 3.5, 4, float("inf")]

# 根據區間範圍計算頻率
filtered_data["selling_price_binned"] = pd.cut(filtered_data["kindle_over_paperback-selling_price"], bins=bins)
filtered_data["original_price_binned"] = pd.cut(filtered_data["kindle_over_paperback-original_price"], bins=bins)

selling_price_counts = filtered_data["selling_price_binned"].value_counts().sort_index()
original_price_counts = filtered_data["original_price_binned"].value_counts().sort_index()

# 設置區間標籤
labels = ["0-0.25", "0.25-0.5", "0.5-0.75", "0.75-1.0", "1.0-1.25", "1.25-1.5", "1.5-1.75", "1.75-2.0", "2.0-2.5", "2.5-3.0", "3.0-3.5", "3.5-4.0", ">4"]

# 創建子圖
fig = make_subplots(rows=1, cols=2, subplot_titles=("電子售價 / 平裝售價", "電子定價 / 平裝定價"), x_title="", y_title="", column_widths=[0.5, 0.5])

# 售價長條圖
fig.add_trace(go.Bar(x=labels, y=selling_price_counts.values, name="Selling Price", marker_color="#1f77b4"), row=1, col=1)

# 原價長條圖
fig.add_trace(go.Bar(x=labels, y=original_price_counts.values, name="Original Price", marker_color="#1f77b4"), row=1, col=2)  # 設置原價長條圖的顏色為藍色

# 設置相同的刻度範圍
max_y = max(selling_price_counts.max(), original_price_counts.max())
buffer = max_y * 0.1  # 預留邊緣緩衝區

fig.update_yaxes(title_font=dict(size=20), tickfont=dict(size=20), range=[0, max_y + buffer], row=1, col=1)
fig.update_yaxes(title_font=dict(size=20), tickfont=dict(size=20), range=[0, max_y + buffer], row=1, col=2)
fig.update_xaxes(tickfont=dict(size=16), title_standoff=10)  # 左子圖
# fig.update_xaxes(tickfont=dict(size=16), row=1, col=2, title_standoff=50)  # 右子圖
fig.update_annotations(font=dict(size=20))
fig.update_layout(xaxis_title_standoff=100, yaxis_title_standoff=50)  # x軸標題的距離  # y軸標題的距離

# 更新子圖佈局
fig.update_layout(height=700, width=1400, title_text="Bar Charts of Kindle Over Paperback Prices", showlegend=False)
fig.update_layout(font=dict(family=r"C:\Users\USER\OneDrive\桌面\Fonts\TaipeiSansTCBeta-Regular.ttf", color="Black"))
fig.update_layout(margin=dict(l=150, r=150, t=100, b=150))
fig.write_image("0801 kindle-paperback/kindle_over_paperback_histogram.svg")
fig.show()

In [14]:
# 創建 DataFrame 儲存售價區間頻率
selling_price_data = pd.DataFrame({"Selling Price Binned": selling_price_counts.index, "Selling Price Count": selling_price_counts.values})

# 創建 DataFrame 儲存原價區間頻率
original_price_data = pd.DataFrame({"Original Price Binned": original_price_counts.index, "Original Price Count": original_price_counts.values})

# 匯出為 Excel，分別放在不同的工作表中
with pd.ExcelWriter("1125/books_com-kindle_over_paperback_price_counts.xlsx") as writer:
    selling_price_data.to_excel(writer, sheet_name="Selling Price Data", index=False)
    original_price_data.to_excel(writer, sheet_name="Original Price Data", index=False)

In [50]:
data_kindle_1dep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2885 entries, 0 to 2884
Data columns (total 14 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   title                                 2885 non-null   object
 1   category                              2885 non-null   object
 2   book_info_link                        2885 non-null   object
 3   kindle-selling_price                  2851 non-null   object
 4   kindle-original_price                 2851 non-null   object
 5   kindle-whole_rank                     2761 non-null   object
 6   paperback-selling_price               1531 non-null   object
 7   paperback-original_price              1537 non-null   object
 8   paperback-whole_rank                  1687 non-null   object
 9   original_kindle_link                  2873 non-null   object
 10  kindle_over_paperback-selling_price   1523 non-null   object
 11  kindle_over_paperback-original

In [43]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def plot_rank_price_ratio(data_con: pd.DataFrame) -> None:
    # 提取相關欄位
    paperback_rank = data_con["paperback-whole_rank"]
    kindle_rank = data_con["kindle-whole_rank"]
    paperback_price = data_con["paperback-selling_price"]
    kindle_price = data_con["kindle-selling_price"]
    title = data_con["title"]

    # 計算價格比例
    price_ratio = kindle_price / paperback_price

    # 剔除缺失值
    valid = ~(paperback_rank.isna() | kindle_rank.isna() | price_ratio.isna())
    paperback_rank = paperback_rank[valid]
    kindle_rank = kindle_rank[valid]
    price_ratio = price_ratio[valid]
    title = title[valid]

    # 計算百分位數
    quantiles = price_ratio.quantile([0, 0.2, 0.4, 0.6, 0.8, 1.0])

    # 分類價格比例
    def get_price_category(ratio):
        if ratio <= quantiles[0.2]:
            return f"0.00~{quantiles[0.2]:.2f}"
        elif quantiles[0.2] < ratio <= quantiles[0.4]:
            return f"{quantiles[0.2]:.2f}~{quantiles[0.4]:.2f}"
        elif quantiles[0.4] < ratio <= quantiles[0.6]:
            return f"{quantiles[0.4]:.2f}~{quantiles[0.6]:.2f}"
        elif quantiles[0.6] < ratio <= quantiles[0.8]:
            return f"{quantiles[0.6]:.2f}~{quantiles[0.8]:.2f}"
        else:
            return f">{quantiles[0.8]:.2f}"

    price_category = price_ratio.apply(get_price_category)

    # 自定義色彩，按範圍大小排序
    color_map = {
        f"0.00~{quantiles[0.2]:.2f}": "purple",
        f"{quantiles[0.2]:.2f}~{quantiles[0.4]:.2f}": "blue",
        f"{quantiles[0.4]:.2f}~{quantiles[0.6]:.2f}": "green",
        f"{quantiles[0.6]:.2f}~{quantiles[0.8]:.2f}": "orange",
        f">{quantiles[0.8]:.2f}": "red",
    }

    # 圖表範圍列表
    ranges = [(0, 4_000_000), (0, 1_000_000), (0, 100_000), (0, 10_000), (0, 1_000)]

    for min_val, max_val in ranges:
        buffer = max_val * 0.05  # 加5%的buffer
        actual_max_val = max_val + buffer

        # 建立子圖
        fig = make_subplots(rows=1, cols=1)

        # 添加主要散點圖
        for category in sorted(color_map.keys()):
            mask = price_category == category
            fig.add_trace(
                go.Scatter(
                    x=paperback_rank[mask],
                    y=kindle_rank[mask],
                    mode="markers",
                    marker=dict(color=color_map[category]),
                    name=category,
                    hovertemplate="Paperback Rank: %{x}<br>kindle Rank: %{y}<br>Price Ratio: %{customdata[0]:.2f}<br>Title: %{customdata[1]}<extra></extra>",
                    customdata=list(zip(price_ratio[mask], title[mask])),
                )
            )

        # 增加 x=y 的直線
        fig.add_trace(go.Scatter(x=[min_val, actual_max_val], y=[min_val, actual_max_val], mode="lines", line=go.scatter.Line(color="black", dash="dash"), showlegend=False))

        # 標示超出範圍的點
        out_of_range = (paperback_rank > max_val) | (kindle_rank > max_val)
        fig.add_trace(
            go.Scatter(
                x=paperback_rank[out_of_range].clip(upper=max_val),
                y=kindle_rank[out_of_range].clip(upper=max_val),
                mode="markers",
                marker=dict(symbol="x", color=price_category[out_of_range].map(color_map)),
                showlegend=False,
                hovertemplate="Paperback Rank: %{x}<br>kindle Rank: %{y}<br>Price Ratio: %{customdata[0]:.2f}<br>Title: %{customdata[1]}<extra></extra>",
                customdata=list(zip(price_ratio[out_of_range], title[out_of_range])),
            )
        )

        # 更新圖表布局
        fig.update_layout(
            height=800,
            width=800,
            title=f"Book Rank Comparison (Rank Range: {min_val} to {max_val})",
            xaxis=dict(title="Paperback Whole Rank", range=[-buffer, actual_max_val], scaleanchor="y", scaleratio=1),
            yaxis=dict(title="kindle Whole Rank", range=[-buffer, actual_max_val], scaleanchor="x", scaleratio=1),
            legend_title="Price Ratio Category",
            legend=dict(traceorder="normal"),
        )

        # 顯示圖表
        fig.write_image(f"0801 kindle-paperback/rank_price_ratio_{min_val}_to_{max_val}.svg")
        fig.show()


# 假設 data_2dep_top100 是你的數據
plot_rank_price_ratio(data_kindle_1dep)

In [15]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def plot_rank_price_ratio(data_con: pd.DataFrame) -> None:
    # 提取相關欄位
    data_con = data_con[data_con["paperback-whole_rank"] <= 100_000]
    data_con = data_con[data_con["kindle-whole_rank"] <= 100_000]
    paperback_rank = data_con["paperback-whole_rank"]
    kindle_rank = data_con["kindle-whole_rank"]
    paperback_price = data_con["paperback-selling_price"]
    kindle_price = data_con["kindle-selling_price"]
    title = data_con["title"]

    # 計算價格比例
    price_ratio = kindle_price / paperback_price

    # 剔除缺失值
    valid = ~(paperback_rank.isna() | kindle_rank.isna() | price_ratio.isna())
    paperback_rank = paperback_rank[valid]
    kindle_rank = kindle_rank[valid]
    price_ratio = price_ratio[valid]
    title = title[valid]

    # 分類價格比例
    def get_price_category(ratio):
        if ratio <= 0.3:
            return f"0.0~0.3"
        elif 0.3 < ratio <= 0.5:
            return f"0.3~0.5"
        elif 0.5 < ratio <= 0.7:
            return f"0.5~0.7"
        elif 0.7 < ratio <= 1.0:
            return f"0.7~1.0"
        else:
            return f">1.0"

    price_category = price_ratio.apply(get_price_category)

    # 自定義色彩，按範圍大小排序
    color_map = {
        f"0.0~0.3": "purple",
        f"0.3~0.5": "blue",
        f"0.5~0.7": "green",
        f"0.7~1.0": "orange",
        f">1.0": "red",
    }

    # 圖表範圍列表
    ranges = [(0, 100_000)]

    for min_val, max_val in ranges:
        buffer = max_val * 0.05
        actual_max_val = max_val + buffer

        # 建立子圖
        fig = make_subplots(rows=1, cols=1)

        # 添加主要散點圖
        for category in sorted(color_map.keys()):
            mask = price_category == category
            fig.add_trace(
                go.Scatter(
                    x=paperback_rank[mask],
                    y=kindle_rank[mask],
                    mode="markers",
                    marker=dict(color=color_map[category]),
                    name=category,
                    hovertemplate="Paperback Rank: %{x}<br>Hardcover Rank: %{y}<br>Price Ratio: %{customdata[0]:.2f}<br>Title: %{customdata[1]}<extra></extra>",
                    customdata=list(zip(price_ratio[mask], title[mask])),
                )
            )

        # 增加 x=y 的直線
        fig.add_trace(go.Scatter(x=[min_val, actual_max_val], y=[min_val, actual_max_val], mode="lines", line=go.scatter.Line(color="black", dash="dash"), showlegend=False))

        # 更新圖表布局
        fig.update_layout(
            height=700,
            width=830,
            title=f"電子與平裝總排名散佈圖",
            xaxis=dict(title="平裝總排名", range=[-buffer, actual_max_val], scaleanchor="y", scaleratio=1),
            yaxis=dict(title="電子總排名", range=[-buffer, actual_max_val], scaleanchor="x", scaleratio=1),
            legend_title="電子相對平裝價格倍數",
            legend=dict(traceorder="normal"),
        )
        fig.update_layout(font=dict(family=r"C:\Users\USER\OneDrive\桌面\Fonts\TaipeiSansTCBeta-Regular.ttf", size=16, color="Black"))
        fig.update_layout(margin=dict(l=100, r=150, t=100, b=100))
        # 顯示圖表
        fig.write_image(f"精裝與平裝總排名散佈圖 - 以精裝相對平裝價格倍數上色.jpg")
        fig.show()
        # 創建匯出的 DataFrame
        colors = price_category.map(color_map)
        export_data = pd.DataFrame({"Paperback Rank": paperback_rank, "Kindle Rank": kindle_rank, "Price Ratio": price_ratio, "Color": colors})

        # 匯出為 Excel
        export_data.to_excel("1125/kindle_paperback_rank_price_ratio.xlsx", index=False)


# 假設 data_2dep_top100 是你的數據
plot_rank_price_ratio(data_kindle_1dep)